In [1]:
import pandas as pd

In [2]:
from xlrd import open_workbook

In [3]:
data1 = open_workbook("OwnActivities.xls")
sheet1 = data1.sheet_by_index(0)    #xls file is accessed

ChEMBLID = []
SMILES1 = []
IC50 = []     #lists for each relevant value created

for k in range(1, sheet1.nrows):
    if sheet1.cell(k,10).value != '' and sheet1.cell(k,8).value == 'IC50':
      IC50.append(sheet1.cell(k,10).value)
      ChEMBLID.append(sheet1.cell(k,0).value)
      SMILES1.append(sheet1.cell(k,7).value)          #for loop checks that compound has an IC50 value, if so adds data to the lists

In [ ]:
SMILES1

In [ ]:
ChEMBLID

In [ ]:
IC50

In [ ]:
print(len(SMILES1)) #to check data has been taken from same number of compounds, prevents issues later
print(len(ChEMBLID))
print(len(IC50))

In [ ]:
pip install rdkit

In [9]:
from rdkit.Chem import AllChem
from rdkit import Chem

In [ ]:
dataframe = pd.DataFrame({'SMILES':SMILES1})     #convert lists to dataframe for algorithm
print(dataframe)
dataframe['IC50'] = IC50
print(dataframe)

In [ ]:
pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
pip install openpyxl

In [ ]:
dataframe.head(1)

In [ ]:
from rdkit import Chem, DataStructs
from rdkit.Chem import PandasTools, AllChem
PandasTools.AddMoleculeColumnToFrame(dataframe,'SMILES','Molecule')
dataframe[["SMILES","Molecule"]].head(1)     #expand dataframe with molecular representation

In [ ]:
dataframe.Molecule.isna().sum()   #checks if any of the SMILES could not be converted

In [17]:
def mol2fp(mol):
    fp = AllChem.GetHashedMorganFingerprint(mol, 2, nBits=4096)
    ar = np.zeros((1,), dtype=np.int8)
    DataStructs.ConvertToNumpyArray(fp, ar)
    return ar   #we define a function which converts an input molecular representation to Morgan Fingerprint

In [18]:
dataframe["FPs"] = dataframe.Molecule.apply(mol2fp)  #use function to convert all compounds to fingerprint

In [ ]:
X = np.stack(dataframe.FPs.values)
print(X.shape)   #shows the number of molecules in the dataset to be analysed and the number of bits in the fingerprint

In [20]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [21]:
y = dataframe.IC50.values.reshape((-1,1))
X_train, X_test, y_train, y_test = train_test_split(X, y,  test_size=0.10, random_state=42)
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train,  test_size=0.05, random_state=42)
#Normalizing output using standard scaling
scaler = StandardScaler()
y_train = scaler.fit_transform(y_train)
y_test = scaler.transform(y_test)
y_validation = scaler.transform(y_validation)   #data is randomly split into train, test and validation sets

In [ ]:
from sklearn.feature_selection import VarianceThreshold
feature_select = VarianceThreshold(threshold=0.05)
X_train = feature_select.fit_transform(X_train)
X_validation = feature_select.transform(X_validation)
X_test = feature_select.transform(X_test) #we remove low variance bits from the fingerprint; they should be insignificant for training and will make learning harder
X_train.shape   #shows number of molecules in training set and number of bits in fingerprint after removal

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)  #transfers to gpu if possible for faster analysis

X_train = torch.tensor(X_train, device=device).float()
X_test = torch.tensor(X_test, device=device).float()
X_validation = torch.tensor(X_validation, device=device).float()
y_train = torch.tensor(y_train, device=device).float()
y_test = torch.tensor(y_test, device=device).float()
y_validation = torch.tensor(y_validation, device=device).float()
X_train    #convert everythig to tensors for pytorch

In [24]:
from torch.utils.data import TensorDataset
train_dataset = TensorDataset(X_train, y_train)
validation_dataset = TensorDataset(X_validation, y_validation)   #turn individual features into dataset

In [25]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=256,
                                          shuffle=True)
validation_loader = torch.utils.data.DataLoader(dataset=validation_dataset,  #prepares datasets to be given to algorithm in batches
                                          batch_size=256,                    #training set shuffled every time to encourage generalising
                                          shuffle=False)                     #validation set does not need shuffling

In [26]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_rate, out_size):
        super(Net, self).__init__()
        # Three layers and a output layer
        self.fc1 = nn.Linear(input_size, hidden_size)  # 1st Full-Connected Layer
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc_out = nn.Linear(hidden_size, out_size) # Output layer
        #Layer normalization for faster training
        self.ln1 = nn.LayerNorm(hidden_size)
        self.ln2 = nn.LayerNorm(hidden_size)
        self.ln3 = nn.LayerNorm(hidden_size)        
        #LeakyReLU will be used as the activation function
        self.activation = nn.LeakyReLU()
        #Dropout for regularization
        self.dropout = nn.Dropout(dropout_rate)
     
    def forward(self, x):# Forward pass: stacking each layer together
        out = self.fc1(x)
        out = self.ln1(out)
        out = self.activation(out)
        out = self.dropout(out)
        out = self.fc2(out)
        out = self.ln2(out)
        out = self.activation(out)
        out = self.dropout(out)
        out = self.fc3(out)
        out = self.ln3(out)
        out = self.activation(out)
        out = self.dropout(out)
        #Final output layer
        out = self.fc_out(out)
        return out

In [27]:
#Defining the hyperparameters
input_size = X_train.size()[-1]     # The input size should be equal to our fingerprint size
hidden_size = 1024   # The size of the hidden layer
dropout_rate = 0.82    # The dropout rate
output_size = 1        # This is just a single task, so this will be one
learning_rate = 0.00005  # The learning rate for the optimizer
model = Net(input_size, hidden_size, dropout_rate, output_size)

In [28]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
model.train() #Ensure the network is in "train" mode with dropouts active
epochs = 200
for e in range(epochs):
    running_loss = 0
    for fps, labels in train_loader:
        # Training pass
        optimizer.zero_grad() # Initialize the gradients, which will be recorded during the forward pass
         
        output = model(fps) #Forward pass of the mini-batch
        loss = criterion(output, labels) #Computing the loss
        loss.backward() # calculate the backward pass
        optimizer.step() # Optimize the weights
         
        running_loss += loss.item()
    else:
        if e%10 == 0:
            validation_loss = torch.mean(( y_validation - model(X_validation) )**2).item()
            print("Epoch: %3i Training loss: %0.2F Validation loss: %0.2F"%(e,(running_loss/len(train_loader)), validation_loss))

In [30]:
model.eval() #Switch to evaluation mode, dropout is switched off
y_pred_train = model(X_train)
y_pred_validation = model(X_validation)
y_pred_test = model(X_test)  #trained model gives predictions for each dataset

In [ ]:
torch.mean(( y_train - y_pred_train )**2).item()

In [ ]:
torch.mean(( y_validation - y_pred_validation )**2).item()

In [ ]:
torch.mean(( y_test - y_pred_test )**2).item()   #mean square error for each set, shows accuracy of model

In [34]:
def flatten(tensor):
    return tensor.cpu().detach().numpy().flatten()  #function turns tensors back into numpy

In [ ]:
plt.scatter(flatten(y_pred_test), flatten(y_test), alpha=0.5, label="Test")
plt.scatter(flatten(y_pred_train), flatten(y_train), alpha=0.1, label="Train")
plt.legend()
plt.plot([-1.5, 1.5], [-1.5,1.5], c="b")   #graph to visualise accuracy of model, can spot odd features

In [ ]:
def predict_smiles(smiles):  #function to predict IC50 of molecule from its SMILES using the model
    fp =mol2fp(Chem.MolFromSmiles(smiles)).reshape(1,-1)  #turn SMILES into representation
    fp_filtered = feature_select.transform(fp)   #filter bits using variance threshold
    fp_tensor = torch.tensor(fp_filtered, device=device).float()  #turn into tensor
    prediction = model(fp_tensor)    #generate prediction of IC50
    IC50 = scaler.inverse_transform(prediction.cpu().detach().numpy())  #convert to numerical output from tensor
    return IC50[0][0]
predict_smiles('CC(=O)Oc1ccccc1C(O)=O')  #current SMILES is for aspririn, replace with desired molecule

Much of the machine learning code was written using Cheminformania 'Building a simple QSAR model using a feed forward neural network in PyTorch' by Esbenbjerrum, so refer to that excellent post if you have further queries